In [73]:
import requests
import json
import pymongo
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [74]:
#mac command to start/stop mongodb
#brew services start mongodb-community@8.0
#brew services stop mongodb-community@8.0

In [75]:
#Create MongoDB Database
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['Spotify']

In [76]:
#To get private details such as playlist, etc. If not this will be more simple
client_id = "8c3ed71b042646858e003a68626f625b"
client_secret = "e9d1ffba94d44a4b8da9b95c1e2e4045"
redirect_uri = "http://localhost:3000"

scope = "playlist-read-private user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret,
                                               redirect_uri=redirect_uri, scope=scope))


In [77]:
playlists = sp.current_user_playlists()
for playlist in playlists['items']:
    if playlist['name'] == "===":
        print(playlist['id'])
        p_id = playlist['id']

3ehU2363fTghRKipKg6dYB


In [78]:
results = sp.playlist_items(p_id)
tracks = results['items']

In [ ]:
#DB Design

songs = [] #for content-based filtering, collection columns: song_id, album_id, artist0,1,..., release_date, duration_ms
song_details = [] #to store song details, collection columns: song_id, song_name, album_id, album_name
singers = [] #to store singer details, collection columns: id, name
artists = {} #To check for keys

for track in tracks:
    collection = {} #entries for database collection
    c = 0

    song_id = track['track']['id']
    album_id = track['track']['album']['id']

    collection['song_id'] = song_id
    collection['album_id'] = album_id

    song_details.append({'song_id': song_id, 'song_name' : track['track']['name'], 'album_id': album_id, 'album_name':  track['track']['album']['name']})

    genres = set()

    for artist in track['track']['album']['artists']:
        collection[f'artist{c+1}'] = artist['id']
        c += 1
        if artist['id'] not in artists.keys():
            artists[artist['id']] = artist['name']
            singers.append({'id':artist['id'] ,'name':artist['name'] }) #append to collection
        
        artist_info = sp.artist(artist['id'])
        genres.update(set(artist_info['genres']))
        
    for idx, genre in enumerate(genres):
        collection[f'genre{idx+1}'] = genre

    collection['release_date'] = track['track']['album']['release_date']
    collection['duration_ms'] = track['track']['duration_ms']

    songs.append(collection)

In [83]:
song_col = db['songs']
song_details_col = db['song_details']
singers_col = db['artists']

In [ ]:
#Run Once
#x = song_col.insert_many(songs)
#y = song_details_col.insert_many(song_details)
#z = singers_col.insert_many(singers)

In [88]:
query = {'song_name':'bad guy'}
doc = song_details_col.find(query)
for i in doc:
    print(i)

{'_id': ObjectId('672caeedf4a77c73b31f4faf'), 'song_id': '2Fxmhks0bxGSBdJ92vM42m', 'song_name': 'bad guy', 'album_id': '0S0KGZnfBGSIssfF54WSJh', 'album_name': 'WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?'}
